# Cruise Point Calculation demo 

This demo shows the application of a cruise point stabilized point calculation.

In [ ]:
from amad.disciplines.design.resources.aircraft_geometry_library import ac_narrow_body_long as airplane_geom
from amad.disciplines.powerplant.systems import EnginePerfoMattingly
import time
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from amad.disciplines.flight_dynamics.systems.crzEquiPoint import CrzEquiPoint
import numpy
import scipy

In [ ]:
"""
Problem Set-up
"""
equi = CrzEquiPoint('equi', asb_aircraft_geometry=airplane_geom(), option_optimization=False, working_directory="amad/disciplines/aerodynamics/temp/avl/")
equi.z_altitude = 8000.8
equi.m_mto = 50000.
equi.mach_current = 0.75

"""
System Performance Calculation
"""
start_time = time.perf_counter()
equi.run_drivers()
end_time = time.perf_counter()
# nb: 15 seconds for this run case
debug_message = (
    f'time={end_time - start_time:.1f} '
    + f'alpha={equi.alpha_aircraft:.5f} '
    + f'l/d={equi.lift_aircraft / equi.drag_aircraft:.1f} '
    + f't_h= {equi.thrust_horizontal:.0f} '
    + f't_v= {equi.thrust_vertical:.0f} '
    + f'lift={equi.lift_aircraft:.0f} '
    + f'drag={equi.drag_aircraft:.0f}'
)
# TODO: print in logging
print(debug_message)

"""
Lift / drag / Delta thrust equilibrium
"""

lift_int = equi.lift_int
drag_int = equi.drag_int

alpha_list = numpy.linspace(-10., 10., 1000)
lift_list = lift_int(alpha_list)
drag_list = drag_int(alpha_list)
th_list = []
tv_list = []
delta_thrust = []
for alpha in alpha_list:
    th = equi._calc_t_h(alpha)
    tv = equi._calc_t_v(alpha)
    th_list.append(th)
    tv_list.append(tv)
    delta_thrust.append(th - tv)

fig = make_subplots(specs=[[{"secondary_y": False}]])

fig.add_trace(
    go.Scatter(x=alpha_list, y=lift_list, name="Lift Force (N)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=alpha_list, y=drag_list, name="Drag Force (N)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=alpha_list, y=th_list, name="Thrust Horizontal Force (N)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=alpha_list, y=tv_list, name="Thrust Vertical Force (N)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=alpha_list, y=delta_thrust, name="Delta Thrust Force (N)"),
    secondary_y=False,
)

fig.update_xaxes(title_text="Alpha (deg)")
fig.update_yaxes(title_text="Force (N)")

fig.update_layout(
    template='plotly_white'
)

fig.show()

"""
Economic Speed Calculation
"""
min_mach = .4
max_mach = .9

mach_list = numpy.linspace(min_mach, max_mach, 6)
mach_list_fine = numpy.linspace(min_mach, max_mach, 1000)
thr_req_list = []
fc_list = []

epm = EnginePerfoMattingly('epm', altitude=equi.z_altitude, dISA=0.)
epm.thrust_eng = 120000

for m in mach_list:
    print(f'calculating thrust for mach {m}')
    equi.mach_current = m
    epm.mach_current = m
    equi.run_drivers()
    epm.run_once()
    thrust = equi.thrust_required
    fc_sec = thrust * epm.SFC  # kg / second
    fc_dist = 1000 * fc_sec / equi.v_tas  # kg / km

    thr_req_list.append(equi.thrust_required)
    fc_list.append(fc_dist)

int_fc = scipy.interpolate.interp1d(mach_list, fc_list, kind='cubic', bounds_error=False)
fc_int_list = [int_fc(m) for m in mach_list_fine]

mach_opt = scipy.optimize.minimize_scalar(int_fc, bounds=(min_mach, max_mach), method='bounded')
print(f'mach optimum = {mach_opt.x:.3f}')

fig2 = make_subplots(specs=[[{"secondary_y": False}]])

fig2.add_trace(
    go.Scatter(x=mach_list_fine, y=fc_int_list, name="Range-specific Fuel Consumption (kg/km)"),
    secondary_y=False,
)

fig2.add_trace(
    go.Scatter(x=[mach_opt.x], y=[int_fc(mach_opt.x)], name="Optimum mach"),
    secondary_y=False,
)

fig2.show()

"""
Lift / drag / debug
"""

lift_int = equi.lift_int
drag_int = equi.drag_int

alpha_list = numpy.linspace(-10., 10., 1000)
lift_list = lift_int(alpha_list)
drag_list = drag_int(alpha_list)

equi.run_drivers()

fig = make_subplots(specs=[[{"secondary_y": False}]])

fig.add_trace(
    go.Scatter(x=alpha_list, y=lift_list, name="Lift Force orig (N)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=alpha_list, y=drag_list, name="Drag Force orig (N)"),
    secondary_y=False,
)

fig.show()